<a href="https://colab.research.google.com/github/zhangxs131/Machine_learning_notebook/blob/main/linear_regresssion_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#线性回归

对于回归问题，最后类别往往是一个连续值进行选择，而非离散值，一般可能需要使用线性回归的方法，这里主要介绍简单线性回归，和局部加权线性回归。

数据集为 鲍鱼年龄预测数据集：https://raw.githubusercontent.com/Jack-Cherish/Machine-Learning/master/Regression/abalone.txt

In [1]:
#下载数据
!wget https://raw.githubusercontent.com/Jack-Cherish/Machine-Learning/master/Regression/abalone.txt

--2022-05-05 02:51:21--  https://raw.githubusercontent.com/Jack-Cherish/Machine-Learning/master/Regression/abalone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 193180 (189K) [text/plain]
Saving to: ‘abalone.txt’

abalone.txt         100%[===================>] 188.65K  --.-KB/s    in 0.02s   

2022-05-05 02:51:21 (7.49 MB/s) - ‘abalone.txt’ saved [193180/193180]



In [4]:
#加载数据
import numpy as np
import matplotlib.pyplot as plt

def load_data(filename):
  with open(filename,'r') as f:
    content=f.readlines()
  

  data,label=[],[]
  for i in content:
    t=i.strip().split('\t')
    data.append([float(j) for j in t[:-1]])
    label.append(float(t[-1]))
  
  return data,label

data,label=load_data('abalone.txt')

In [8]:
#局部加权线性回归实现

def lwlr(testPoint,data,label,k=1.0):
  #k为高斯核的k值，k越小越拟合训练集

  xMat=np.mat(data)
  yMat=np.mat(label).T
  m=xMat.shape[0]
  weights=np.mat(np.eye((m))) #创建对焦矩阵

  for j in range(m):
    diffMat=testPoint-xMat[j,:]
    weights[j,j]=np.exp(diffMat*diffMat.T/(-2.0*k**2))
  
  xTx=xMat.T*(weights*xMat)
  if np.linalg.det(xTx)==0.0:
    return 
  ws=xTx.I *(xMat.T*(weights*yMat))

  return testPoint*ws

def lwlrTest(testArr,data,label,k=1.0):
  testArr=np.mat(testArr)
  m=testArr.shape[0]
  yHat=np.zeros(m)
  for i in range(m):
    yHat[i]=lwlr(testArr[i],data,label,k)
  return yHat

def standRegres(xArr,yArr):
    xMat = np.mat(xArr); yMat = np.mat(yArr).T
    xTx = xMat.T * xMat                            #根据文中推导的公示计算回归系数
    if np.linalg.det(xTx) == 0.0:
        print("矩阵为奇异矩阵,不能求逆")
        return
    ws = xTx.I * (xMat.T*yMat)
    return ws

  
def rssError(yArr,yHatArr):
  return ((yArr-yHatArr)**2).sum()

abX,abY=data,label
print('训练集与测试集相同:局部加权线性回归,核k的大小对预测的影响:')
yHat01 = lwlrTest(abX[0:99], abX[0:99], abY[0:99], 0.1)
yHat1 = lwlrTest(abX[0:99], abX[0:99], abY[0:99], 1)
yHat10 = lwlrTest(abX[0:99], abX[0:99], abY[0:99], 10)
print('k=0.1时,误差大小为:',rssError(abY[0:99], yHat01.T))
print('k=1  时,误差大小为:',rssError(abY[0:99], yHat1.T))
print('k=10 时,误差大小为:',rssError(abY[0:99], yHat10.T))
print('')
print('训练集与测试集不同:局部加权线性回归,核k的大小是越小越好吗？更换数据集,测试结果如下:')
yHat01 = lwlrTest(abX[100:199], abX[0:99], abY[0:99], 0.1)
yHat1 = lwlrTest(abX[100:199], abX[0:99], abY[0:99], 1)
yHat10 = lwlrTest(abX[100:199], abX[0:99], abY[0:99], 10)
print('k=0.1时,误差大小为:',rssError(abY[100:199], yHat01.T))
print('k=1  时,误差大小为:',rssError(abY[100:199], yHat1.T))
print('k=10 时,误差大小为:',rssError(abY[100:199], yHat10.T))
print('')
print('训练集与测试集不同:简单的线性归回与k=1时的局部加权线性回归对比:')
print('k=1时,误差大小为:', rssError(abY[100:199], yHat1.T))
ws = standRegres(abX[0:99], abY[0:99])
yHat = np.mat(abX[100:199]) * ws
print('简单的线性回归误差大小:', rssError(abY[100:199], yHat.T.A))

训练集与测试集相同:局部加权线性回归,核k的大小对预测的影响:
k=0.1时,误差大小为: 56.78868743048742
k=1  时,误差大小为: 429.8905618704059
k=10 时,误差大小为: 549.1181708828803

训练集与测试集不同:局部加权线性回归,核k的大小是越小越好吗？更换数据集,测试结果如下:
k=0.1时,误差大小为: 57913.51550155909
k=1  时,误差大小为: 573.5261441894984
k=10 时,误差大小为: 517.5711905381573

训练集与测试集不同:简单的线性归回与k=1时的局部加权线性回归对比:
k=1时,误差大小为: 573.5261441894984
简单的线性回归误差大小: 518.6363153245542
